This is the new ligand clustering and contacts file (01/16/2025)

In [64]:
#this script will find the center of mass for each ligand pose
#then it will cluster the poses based on the center of mass (KNN or distance based)
#then it will write out the poses for each cluster into one PDB file per cluster
# ChatGPT was used to generate parts of the script

#1. Do one frame per ligand
#2. Get center of mass of each ligand
#3. Do the clustering
#4. Do the contacts
    #4A. Load the trajectory of one cluster.
    #4B. Get all possible atom pairs (all protein CA – all LIG heavy atoms )
    #4C. Get all distances (for every possible atom pairs); this will give you a list of distances for every frame/model
    #4D. Iterate over the distances (so you get all the distances in a frame)
        #i. select all pairs within a certain distance
        #ii. get the atom objects
        #iii. write the line

In [65]:
# Got rid of:atp_cluster_29, atp_cluster_32, atp_cluster_9 from SwissDock because they were either (A) not binding to a likely spot on the protein or (B) the only binding pose in that area

In [66]:
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import os
import mdtraj as md

In [67]:
# Define output folder
output_folder = '/Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output'

In [68]:
# Expanded dictionary of atomic weights for common elements
atomic_weights = {
    "H": 1.008,
    "C": 12.011,
    "N": 14.007,
    "O": 15.999,
    "P": 30.974,
    "S": 32.06,
    "CL": 35.45,
    "K": 39.098,
    "CA": 40.078,
    "FE": 55.845,
    "ZN": 65.38,
    # Add more elements as needed
}

# Define cut off distance for contact
cutoff_distance = 0.5 # in nm

In [124]:
# Define pathways of the AMBER minimized structure and the PDB file from SwissDock with all the poses
amber_minimized_structure = "/Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output/input/complex_noH.pdb" #Undocked
swissdock_poses = "/Users/stuytschaevers/Desktop/Thesis/atp_mg2+/swissDock/combined/TrwD_atp_FINAL.pdb" # Docked poses from SwissDock
# swissdock_poses = "/Users/stuytschaevers/Desktop/Thesis/atp_mg2+/swissDock/combined/TrwD_atp.pdb" # Docked poses from SwissDock

In [70]:
# Use /Users/stuytschaevers/Desktop/Thesis/SwissDock/scripts/cluster_separator.py to get separate ligand files
# Define the pathway for the PDB file containing all the poses: /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/swissDock/clusters.dock4.pdb
# Define the pathway for the output file for each individual PDB file of each pose: /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/swissDock/clusters_all
# Define the ligand name: atp
cluster_dir = "/Users/stuytschaevers/Desktop/Thesis/atp_mg2+/swissDock/clusters_final"
swissdock_prot = "/Users/stuytschaevers/Desktop/Thesis/atp_mg2+/swissDock/target.pdb"

# Got rid of clusters 2, 19, 28 because they were either (A) not binding to a likely spot on the protein or (B) the only binding pose in that area

In [71]:
num_atoms = 0
all_trajs = []  # List to store all ligand trajectories

# Loop through all files in the directory
for file in os.listdir(cluster_dir):
    if file.endswith(".pdb"):
        # Full path to the file
        file_path = os.path.join(cluster_dir, file)
        
        # Load the PDB file as an MDTraj trajectory
        traj = md.load(file_path)
        print(f"Loaded {file}")
        
        # Filter to get all LIG residues
        ligand_residues = [res for res in traj.topology.residues if res.name == 'LIG']

        if ligand_residues:
            for i, res in enumerate(ligand_residues):
                num_atoms = len([atom for atom in res.atoms])
            
            # Reshape the XYZ coordinates to separate each ligand
            XYZ = traj.xyz[0]  # Get the coordinates of all molecules in the first (and only) frame
            print(f"Shape before reshaping: {np.shape(XYZ)}")
            XYZ = XYZ.reshape(-1, num_atoms, 3)  # Reshape into arrays of N atoms
            print(f"Shape after reshaping: {np.shape(XYZ)}")
            
            # Creates trajectory where each frame is a ligand
            ligand_slice = traj.top.select(f"resid {ligand_residues[0].index}")
            ligand_top = traj.atom_slice(ligand_slice).topology
            ligand_trajs = [md.Trajectory(xyz=np.array([xyz]), topology=ligand_top) for xyz in XYZ]
            
            # Store all trajectories from this file
            all_trajs.extend(ligand_trajs)
            
        else:
            print(f"No ligands found in {file}")

# At this point, all_trajs contains all the ligand trajectories from all files
print(f"Total number of ligand trajectories stored: {len(all_trajs)}")

# Load the protein
protein = md.load(swissdock_prot)

# List to hold the stacked ligand-protein trajectories (each ligand in a separate frame)
stacked_trajectory_frames = []

# Loop through all ligand trajectories and stack with protein
for t in all_trajs:  # Process each ligand trajectory
    for frame_lig in t:
        # Stack the protein to every ligand frame and add to the list of frames
        stacked_trajectory_frames.append(protein.stack(frame_lig))

# Combine all stacked frames into a single trajectory
final_trajectory = md.join(stacked_trajectory_frames)

# Save the final trajectory where each frame is a unique ligand stacked with the protein
final_trajectory.save_pdb(f"{output_folder}/ligand_protein_combined.pdb")
print(f"Saved combined PDB with all ligands as separate frames: ligand_protein_combined.pdb")

Loaded atp_cluster_0.pdb
Shape before reshaping: (43, 3)
Shape after reshaping: (1, 43, 3)
Loaded atp_cluster_1.pdb
Shape before reshaping: (43, 3)
Shape after reshaping: (1, 43, 3)
Loaded atp_cluster_3.pdb
Shape before reshaping: (43, 3)
Shape after reshaping: (1, 43, 3)
Loaded atp_cluster_6.pdb
Shape before reshaping: (43, 3)
Shape after reshaping: (1, 43, 3)
Loaded atp_cluster_7.pdb
Shape before reshaping: (43, 3)
Shape after reshaping: (1, 43, 3)
Loaded all_clusters_FINAL.pdb
Shape before reshaping: (215, 3)
Shape after reshaping: (5, 43, 3)
Loaded atp_cluster_5.pdb
Shape before reshaping: (43, 3)
Shape after reshaping: (1, 43, 3)
Loaded atp_cluster_4.pdb
Shape before reshaping: (43, 3)
Shape after reshaping: (1, 43, 3)
Loaded atp_cluster_20.pdb
Shape before reshaping: (43, 3)
Shape after reshaping: (1, 43, 3)
Loaded atp_cluster_21.pdb
Shape before reshaping: (43, 3)
Shape after reshaping: (1, 43, 3)
Loaded atp_cluster_23.pdb
Shape before reshaping: (43, 3)
Shape after reshaping: (

In [72]:
# Define list to store the center of mass for each ligand
ligand_coms = []

# Calculate the center of mass for each ligand
for frame in final_trajectory:
    # Calculate center of mass for each ligand
    com = md.compute_center_of_mass(frame)[0] # Extract COM for the single frame in the trajectory
    ligand_coms.append(com)

# Convert list of COMs to Numpy array (should be a 2D array: n_ligands X 3)
ligand_coms = np.array(ligand_coms)

# Print shape of the COM array
print(f"Shape of the ligand COM array: {ligand_coms.shape}") # Should be (n_ligands, 3)

Shape of the ligand COM array: (41, 3)


In [73]:
# Calculate the pairwise differences in COM of each ligand
# This will be used to cluster the ligands
# This will create a matrix
diffs_com = cdist(ligand_coms, ligand_coms)

# Print the pairwise differences in COM
print("Pairwise COM differences: " + "\n" + str(diffs_com))

Pairwise COM differences: 
[[0.         0.00154718 0.00100775 ... 0.00221077 0.00889925 0.00204719]
 [0.00154718 0.         0.0016535  ... 0.00356298 0.00909188 0.00355461]
 [0.00100775 0.0016535  0.         ... 0.00304932 0.0079296  0.00266134]
 ...
 [0.00221077 0.00356298 0.00304932 ... 0.         0.01033316 0.00085132]
 [0.00889925 0.00909188 0.0079296  ... 0.01033316 0.         0.00956467]
 [0.00204719 0.00355461 0.00266134 ... 0.00085132 0.00956467 0.        ]]


In [74]:
# Cluster the ligands using KMeans
from sklearn.cluster import KMeans

# Example: Using K-Means for clustering
# Define the K-Means model with 2 clusters
kmeans = KMeans(n_clusters=2, random_state=42)

# Fit the model to the pairwise differences in COM
kmeans.fit(diffs_com)

# Get the cluster labels
cluster_labels = kmeans.labels_

# Print the cluster labels
print("Cluster labels:", cluster_labels)

# Analyze the results
for cluster in np.unique(cluster_labels):
    print(f"Cluster {cluster}: {np.sum(cluster_labels == cluster)} poses")

# Checked this and it aligns with the visual clustering after opening in ChimeraX
# Print cluster number and the cluster label 
#for i, label in enumerate(cluster_labels):
#    print(f"Cluster number: {i}, Cluster label: {label}")

Cluster labels: [0 0 0 0 0 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 1 0 0 0 1 0 0 1 1 1 1 0 0 0 0
 1 0 0 0]
Cluster 0: 26 poses
Cluster 1: 15 poses


In [75]:
# Save individual PDB files for each cluster
# Note: this includes the protein as well
for cluster in np.unique(cluster_labels):
    
    # Get the indices of ligands in this cluster
    cluster_indices = np.where(cluster_labels == cluster)[0]
    
    # Extract the frames of ligands in this cluster
    cluster_frames = [all_trajs[idx][0] for idx in cluster_indices]
    
    # Combine the cluster frames with the protein
    cluster_stacked_frames = []
    for frame in cluster_frames:
        cluster_stacked_frames.append(protein.stack(frame))
    
    # Combine all stacked frames into a single trajectory
    cluster_trajectory = md.join(cluster_stacked_frames)

   # Ensure the "cluster" folder exists inside the output folder
    cluster_folder = os.path.join(output_folder, "clusters")
    os.makedirs(cluster_folder, exist_ok=True)

   # Save the cluster trajectory as a PDB file in the single "cluster" folder
    cluster_file = os.path.join(cluster_folder, f"cluster_{cluster}.pdb")
    cluster_trajectory.save_pdb(cluster_file)

    # Print the number of frames saved and the file location
    print(f"Saved {len(cluster_trajectory)} frames to {cluster_file}")

Saved 26 frames to /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output/clusters/cluster_0.pdb
Saved 15 frames to /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output/clusters/cluster_1.pdb


In [76]:
# Create lists to store protein and ligand information for each cluster
protein_ca_counts = {}
ligand_heavy_counts = {}

# Loop through all cluster PDB files in the directory
for cluster_file in sorted(os.listdir(cluster_folder)):
    if cluster_file.endswith(".pdb") and "cluster" in cluster_file:
        # Load the cluster PDB file
        cluster_path = os.path.join(cluster_folder, cluster_file)
        cluster_traj = md.load(cluster_path)
        
        # Select CA atoms of the protein
        protein_ca_indices = cluster_traj.top.select("name CA and protein")
        protein_ca = cluster_traj.atom_slice(protein_ca_indices)
        num_protein_ca = protein_ca.n_atoms
        
        # Select heavy atoms of the ligand
        ligand_heavy_indices = cluster_traj.top.select("resname LIG and not element H")
        ligand_heavy = cluster_traj.atom_slice(ligand_heavy_indices)
        num_ligand_heavy = ligand_heavy.n_atoms
        
        # Store results in dictionaries
        cluster_name = cluster_file.replace(".pdb", "")
        protein_ca_counts[cluster_name] = num_protein_ca
        ligand_heavy_counts[cluster_name] = num_ligand_heavy

        # Print summary for this cluster
        print(f"Processed {cluster_name}:")
        print(f"  Number of CA atoms in protein: {num_protein_ca}")
        print(f"  Number of heavy atoms in ligands: {num_ligand_heavy}\n")

# Summary of all clusters
print("Summary of all clusters:")
for cluster_name in protein_ca_counts.keys():
    print(f"{cluster_name}: Protein CA atoms = {protein_ca_counts[cluster_name]}, Ligand heavy atoms = {ligand_heavy_counts[cluster_name]}")

Processed all_combined_clusters:
  Number of CA atoms in protein: 358
  Number of heavy atoms in ligands: 31

Processed all_combined_clusters_sorted:
  Number of CA atoms in protein: 358
  Number of heavy atoms in ligands: 31

Processed cluster_0:
  Number of CA atoms in protein: 358
  Number of heavy atoms in ligands: 31

Processed cluster_1:
  Number of CA atoms in protein: 358
  Number of heavy atoms in ligands: 31

Summary of all clusters:
all_combined_clusters: Protein CA atoms = 358, Ligand heavy atoms = 31
all_combined_clusters_sorted: Protein CA atoms = 358, Ligand heavy atoms = 31
cluster_0: Protein CA atoms = 358, Ligand heavy atoms = 31
cluster_1: Protein CA atoms = 358, Ligand heavy atoms = 31


In [77]:
# Load both clusters
cluster0 = md.load(f"{output_folder}/clusters/cluster_0.pdb")
cluster1 = md.load(f"{output_folder}/clusters/cluster_1.pdb")

# Combine the two clusters (by stacking trajectories)
combined_clusters = md.join([cluster0, cluster1])

# Save the combined clusters into a single PDB file
combined_clusters.save_pdb(f"{output_folder}/clusters/all_combined_clusters.pdb")

In [78]:
# Load all combined trajectory
all_combined = md.load(f"{output_folder}/clusters/all_combined_clusters.pdb")

In [79]:
# Get frame index for each pose in each cluster
frame_indices = []

# Loop through all frames in the combined clusters
for i in range(len(combined_clusters)):
    # Get the cluster number based on the frame index
    cluster_num = 0 if i < len(cluster0) else 1
    
    # Get the pose index within the cluster
    pose_index = i if cluster_num == 0 else i - len(cluster0)
    
    # Store the cluster number and pose index
    frame_indices.append((cluster_num, pose_index))

# Print the frame indices for each pose
#for i, (cluster_num, pose_index) in enumerate(frame_indices):
#    print(f"Frame {i}: Cluster {cluster_num}, Pose {pose_index}")

In [80]:
# This gives you the original pose # from SwissDock

# Step 1: Get unique clusters
unique_clusters = np.unique(cluster_labels)
print("Unique clusters found:", unique_clusters)

# Step 2: Create a mapping of clusters to frame indices
cluster_to_frames = {cluster: np.where(cluster_labels == cluster)[0] for cluster in unique_clusters}

# Step 3: Print frames for each cluster, along with the frame number
for cluster, indices in cluster_to_frames.items():
    print(f"Cluster {cluster}:")
    for idx in indices:
        frame_number = idx + 1  # Frame numbers are typically 1-based
        print(f"  Frame index: {idx}")

# Example: Access frames for each cluster
for cluster in unique_clusters:
    frames = [all_trajs[idx] for idx in cluster_to_frames[cluster]]
    print(f"Cluster {cluster} frames count:", len(frames))

# Label the frame indices with the cluster number
# This will be useful for later analysis
frame_labels = np.zeros(len(cluster_labels), dtype=int)
for cluster, indices in cluster_to_frames.items():
    frame_labels[indices] = cluster

# Print the frame labels
print("Frame labels:")
print(frame_labels)

Unique clusters found: [0 1]
Cluster 0:
  Frame index: 0
  Frame index: 1
  Frame index: 2
  Frame index: 3
  Frame index: 4
  Frame index: 5
  Frame index: 6
  Frame index: 10
  Frame index: 11
  Frame index: 13
  Frame index: 15
  Frame index: 16
  Frame index: 18
  Frame index: 19
  Frame index: 23
  Frame index: 24
  Frame index: 25
  Frame index: 27
  Frame index: 28
  Frame index: 33
  Frame index: 34
  Frame index: 35
  Frame index: 36
  Frame index: 38
  Frame index: 39
  Frame index: 40
Cluster 1:
  Frame index: 7
  Frame index: 8
  Frame index: 9
  Frame index: 12
  Frame index: 14
  Frame index: 17
  Frame index: 20
  Frame index: 21
  Frame index: 22
  Frame index: 26
  Frame index: 29
  Frame index: 30
  Frame index: 31
  Frame index: 32
  Frame index: 37
Cluster 0 frames count: 26
Cluster 1 frames count: 15
Frame labels:
[0 0 0 0 0 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 1 0 0 0 1 0 0 1 1 1 1 0 0 0 0
 1 0 0 0]


In [81]:
# Sort the frames based on the cluster number
sorted_indices = np.argsort(frame_labels)
sorted_trajectory = all_combined[sorted_indices]

# Save the sorted trajectory
sorted_trajectory.save_pdb(f"{output_folder}/clusters/all_combined_clusters_sorted.pdb")

In [82]:
# Get CA atoms of protein for each PDB file
def get_CA_traj(traj):
    # Select CA atoms
    ca_atoms = traj.topology.select("name CA and protein")
    return traj.atom_slice(ca_atoms)

# Get non-H atoms of ligand for each PDB file
def get_lig_traj(traj, lig_name):
    # Select non-H atoms
    lig_atoms = traj.top.select(f"resname {lig_name} and not element H")
    return traj.atom_slice(lig_atoms)

In [83]:
# Load each cluster file and get CA atoms of protein and non-H atoms of ligand
cluster0 = md.load(f"{output_folder}/clusters/cluster_0.pdb")
cluster1 = md.load(f"{output_folder}/clusters/cluster_1.pdb")

# Get CA atoms of protein for each cluster
cluster0_ca_traj = get_CA_traj(cluster0)
cluster1_ca_traj = get_CA_traj(cluster1)

# Get non-H atoms of ligand for each cluster
cluster0_lig_traj = get_lig_traj(cluster0, "LIG")
cluster1_lig_traj = get_lig_traj(cluster1, "LIG")

# Print the number of CA atoms and ligand atoms for each cluster
print("Cluster 0:")
print(f"  Number of CA atoms: {cluster0_ca_traj.n_atoms}")
print(f"  Number of ligand atoms: {cluster0_lig_traj.n_atoms}")

print("Cluster 1:")
print(f"  Number of CA atoms: {cluster1_lig_traj.n_atoms}")
print(f"  Number of ligand atoms: {cluster1_lig_traj.n_atoms}")

print(type(cluster0_lig_traj))
print(type(cluster0_lig_traj.topology.select("name CA and protein")))

Cluster 0:
  Number of CA atoms: 358
  Number of ligand atoms: 31
Cluster 1:
  Number of CA atoms: 31
  Number of ligand atoms: 31
<class 'mdtraj.core.trajectory.Trajectory'>
<class 'numpy.ndarray'>


In [84]:
# Function to get protein CA atoms
def get_protein_ca_atoms(traj):
    return traj.top.select("name CA and protein")

# Function to get ligand heavy atoms
def get_ligand_heavy_atoms(traj, lig_name):
    return traj.top.select(f"resname {lig_name} and not element H")

cluster0_ca_atoms = get_protein_ca_atoms(cluster0)
cluster0_ligand_atoms = get_ligand_heavy_atoms(cluster0, "LIG")
cluster1_ca_atoms = get_protein_ca_atoms(cluster1)
cluster1_ligand_atoms = get_ligand_heavy_atoms(cluster1, "LIG")

In [85]:
# Function to get pairs of CA protein atoms and ligand heavy atoms
def get_CA_lig_pairs(cluster_traj, cluster_ca_atoms, cluster_lig_atoms):

    # Get all possible pairs of CA protein atoms and ligand heavy atoms
    pairs = cluster_traj.top.select_pairs(cluster_ca_atoms, cluster_lig_atoms)
    return pairs

# Get pairs of CA protein atoms and ligand heavy atoms for each cluster
cluster0_pairs = get_CA_lig_pairs(cluster0, cluster0_ca_atoms, cluster0_ligand_atoms)
cluster1_pairs = get_CA_lig_pairs(cluster1, cluster1_ca_atoms, cluster1_ligand_atoms)

# Print the number of pairs for each cluster
print("Cluster 0:")
print(f"  Number of CA-ligand pairs: {len(cluster0_pairs)}")
print("Cluster 1:")
print(f"  Number of CA-ligand pairs: {len(cluster1_pairs)}")

Cluster 0:
  Number of CA-ligand pairs: 11098
Cluster 1:
  Number of CA-ligand pairs: 11098


In [86]:
# Function to compute distance between protein Ca and ligand heavy atoms
def compute_distances(traj, pairs):
    return md.compute_distances(traj, pairs)

# Get distances for each cluster
distances_cluster0 = compute_distances(cluster0, cluster0_pairs)
distances_cluster1 = compute_distances(cluster1, cluster1_pairs)

# Print shape of each
print(f"Shape of distances array for cluster 0: {distances_cluster0.shape}")
print(f"Shape of distances array for cluster 0: {distances_cluster1.shape}")

Shape of distances array for cluster 0: (26, 11098)
Shape of distances array for cluster 0: (15, 11098)


In [87]:
# Function get contacts that meet the cutoff
def process_cluster_contacts(traj, distances, pairs, cutoff_distance=0.5):
    # Define the contact variable
    contacts = []

    for dist in distances: 
        check = dist < cutoff_distance  # Check if distance is less than cutoff
        tmp_pairs = pairs[check]       # Get pairs that meet the cutoff
        tmp_distances = dist[check]    # Get distances that meet the cutoff

        for pair, distance in zip(tmp_pairs, tmp_distances):
            a1 = traj.top.atom(pair[0])  # Get atom 1
            a2 = traj.top.atom(pair[1])  # Get atom 2

            contacts.append([
                a1.residue.resSeq, a1.residue.name, a1.name,
                a2.residue.resSeq, a2.residue.name, a2.name,
                distance
            ])
            
            # Print the contact info
            print(a1.residue.resSeq, a1.residue.name, a1.name, 
                  a2.residue.resSeq, a2.residue.name, a2.name, distance)

    # Convert contacts to a numpy array after the loop completes
    contacts_np = np.array(contacts, dtype=object)
    return contacts_np

# Process all clusters
all_contacts_cluster0 = process_cluster_contacts(cluster0, distances_cluster0, cluster0_pairs,cutoff_distance)
all_contacts_cluster1 = process_cluster_contacts(cluster1, distances_cluster1, cluster1_pairs,cutoff_distance)

13 GLY CA 1 LIG C6 0.48727563
13 GLY CA 1 LIG N6 0.39905682
14 ASN CA 1 LIG C6 0.43232545
14 ASN CA 1 LIG N1 0.4007872
14 ASN CA 1 LIG C2 0.48034942
14 ASN CA 1 LIG N6 0.41062653
71 GLN CA 1 LIG N6 0.43277842
14 ASN CA 1 LIG C6 0.46571764
14 ASN CA 1 LIG N1 0.4384165
14 ASN CA 1 LIG C2 0.45740402
85 PRO CA 1 LIG O1A 0.4220749
86 LYS CA 1 LIG PA 0.472602
86 LYS CA 1 LIG O1A 0.3964908
86 LYS CA 1 LIG O5* 0.42553732
13 GLY CA 1 LIG C5 0.4791704
13 GLY CA 1 LIG C6 0.38470137
13 GLY CA 1 LIG N1 0.33968508
13 GLY CA 1 LIG C2 0.39897874
13 GLY CA 1 LIG N3 0.48576632
13 GLY CA 1 LIG N6 0.38642055
14 ASN CA 1 LIG N9 0.46358404
14 ASN CA 1 LIG C8 0.46042484
14 ASN CA 1 LIG N7 0.40773237
14 ASN CA 1 LIG C5 0.39560223
14 ASN CA 1 LIG C6 0.4046551
14 ASN CA 1 LIG N1 0.4708147
14 ASN CA 1 LIG C4 0.44697183
14 ASN CA 1 LIG N6 0.4011761
71 GLN CA 1 LIG C6 0.45498455
71 GLN CA 1 LIG N1 0.48718736
71 GLN CA 1 LIG N6 0.39668396
72 ALA CA 1 LIG N6 0.4453973
75 THR CA 1 LIG N7 0.49540848
86 LYS CA 1 LIG C5

In [88]:
# Get information on the contact list
print(type(all_contacts_cluster0))

print(all_contacts_cluster0)

print(len(all_contacts_cluster0))

<class 'numpy.ndarray'>
[[13 'GLY' 'CA' ... 'LIG' 'C6' 0.48727563]
 [13 'GLY' 'CA' ... 'LIG' 'N6' 0.39905682]
 [14 'ASN' 'CA' ... 'LIG' 'C6' 0.43232545]
 ...
 [86 'LYS' 'CA' ... 'LIG' 'C5' 0.41712332]
 [86 'LYS' 'CA' ... 'LIG' 'C6' 0.48070264]
 [86 'LYS' 'CA' ... 'LIG' 'C4' 0.44452676]]
391


In [89]:
def write_contacts_to_file(contacts, filename):
    # Write the contacts to a file
    with open(filename, 'w') as f:
        chain_counter = 1  # Counter for chain IDs
        old_residue_index = -1  # Initialize the previous residue index

        for contact in contacts:
            # Extract contact information
            res1, res_name1, atom_name1, res2, res_name2, atom_name2, distance = contact
            
            # Chain IDs for formatting (modify as needed)
            chain_id_a = "A"
            chain_id_b = "B"

            lig_res_name = "ATP"  # Ligand residue name

            # Format string for the pseudobond file
            #string = f"#1.{chain_id_a}:{res1}@{atom_name1} #1.{chain_id_b}:{res2}@{atom_name2}\n"
            #string = f"#1.{chain_id_a}:{res1}@{atom_name1} #1.{chain_id_b}:LIG@{atom_name2}\n"
            #string = f"#1.{chain_counter}/{chain_id_a}:{res1}@{atom_name1} #1.{chain_counter}/{chain_id_b}:{lig_res_name}@{atom_name2} {distance}\n"
            string = f"/{chain_id_a}:{res1}@{atom_name1} /{chain_id_b}:{lig_res_name}@{atom_name2} {distance}\n"

            # Write the formatted string to the file
            f.write(string)

            # Check if we need to increment the chain counter
            if res1 < old_residue_index:
                chain_counter += 1
            old_residue_index = res1  # Update the previous residue index     

# Save contacts to files
write_contacts_to_file(all_contacts_cluster0, f"{output_folder}/contacts_cluster0.pb")
write_contacts_to_file(all_contacts_cluster1, f"{output_folder}/contacts_cluster1.pb")

In [90]:
def write_contacts_to_file_visualization(contacts, filename):
    # Write the contacts to a file
    with open(filename, 'w') as f:
        chain_counter = 1  # Counter for chain IDs
        old_residue_index = -1  # Initialize the previous residue index

        for contact in contacts:
            # Extract contact information
            res1, res_name1, atom_name1, res2, res_name2, atom_name2, distance = contact
            
            # Chain IDs for formatting (modify as needed)
            chain_id_a = "A"
            chain_id_b = "B"

            lig_res_name = "LIG"  # Ligand residue name

            # Format string for the pseudobond file
            #string = f"#1.{chain_id_a}:{res1}@{atom_name1} #1.{chain_id_b}:{res2}@{atom_name2}\n"
            #string = f"#1.{chain_id_a}:{res1}@{atom_name1} #1.{chain_id_b}:LIG@{atom_name2}\n"
            string = f"#1.{chain_counter}/{chain_id_a}:{res1}@{atom_name1} #1.{chain_counter}/{chain_id_b}:{lig_res_name}@{atom_name2} \n"
            #string = f"/{chain_id_a}:{res1}@{atom_name1} /{chain_id_b}:{lig_res_name}@{atom_name2} {distance}\n"

            # Write the formatted string to the file
            f.write(string)

            # Check if we need to increment the chain counter
            if res1 < old_residue_index:
                chain_counter += 1
            old_residue_index = res1  # Update the previous residue index     

# Save contacts to files
write_contacts_to_file_visualization(all_contacts_cluster0, f"{output_folder}/contacts_cluster0_visualization.pb")
write_contacts_to_file_visualization(all_contacts_cluster1, f"{output_folder}/contacts_cluster1_visualization.pb")

In [91]:
# This function gets the number of contacts per pose
def get_number_of_contacts_per_pose(all_contacts):
    chain_counter = 1 # Counter for chain IDs
    old_residue_index = -1 # Initialize the previous residue index
    contacts_per_pose = {} # Dictionary to store contacts per pose
    
    for contact in all_contacts:
        # Extract contact information
        res1, res_name1, atom_name1, res2, res_name2, atom_name2, distance = contact
        
        # Check if the residue index is in the dictionary
        if res1 not in contacts_per_pose:
            contacts_per_pose[res1] = 1
        else:
            contacts_per_pose[res1] += 1
        
        # Check if we need to increment the chain counter
        if res1 < old_residue_index:
            chain_counter += 1
        old_residue_index = res1

        # Count the number of contacts per pose
        if chain_counter not in contacts_per_pose:
            contacts_per_pose[chain_counter] = 0
        contacts_per_pose[chain_counter] += 1
    return contacts_per_pose

# Get the number of contacts per pose for each cluster
contacts_per_pose_cluster0 = get_number_of_contacts_per_pose(all_contacts_cluster0)
contacts_per_pose_cluster1 = get_number_of_contacts_per_pose(all_contacts_cluster1)

print("Contacts per pose for cluster 0:")
for chain, count in contacts_per_pose_cluster0.items():
    print(f"Pose {chain}: {count} contacts")

print("Contacts per pose for cluster 1:")
for chain, count in contacts_per_pose_cluster1.items():
    print(f"Pose {chain}: {count} contacts")


Contacts per pose for cluster 0:
Pose 13: 112 contacts
Pose 1: 7 contacts
Pose 14: 125 contacts
Pose 71: 19 contacts
Pose 2: 7 contacts
Pose 85: 13 contacts
Pose 86: 94 contacts
Pose 3: 22 contacts
Pose 72: 1 contacts
Pose 75: 11 contacts
Pose 4: 15 contacts
Pose 5: 31 contacts
Pose 84: 28 contacts
Pose 87: 1 contacts
Pose 89: 4 contacts
Pose 90: 14 contacts
Pose 6: 13 contacts
Pose 7: 13 contacts
Pose 8: 11 contacts
Pose 9: 12 contacts
Pose 74: 2 contacts
Pose 10: 13 contacts
Pose 11: 18 contacts
Pose 12: 22 contacts
Pose 15: 18 contacts
Pose 103: 2 contacts
Pose 104: 2 contacts
Pose 16: 49 contacts
Pose 17: 14 contacts
Pose 18: 8 contacts
Pose 19: 16 contacts
Pose 20: 16 contacts
Pose 21: 20 contacts
Pose 22: 4 contacts
Pose 23: 10 contacts
Pose 24: 15 contacts
Contacts per pose for cluster 1:
Pose 339: 2 contacts
Pose 1: 2 contacts
Pose 340: 2 contacts
Pose 162: 4 contacts
Pose 2: 14 contacts
Pose 163: 4 contacts
Pose 179: 13 contacts
Pose 180: 39 contacts
Pose 209: 6 contacts
Pose 

In [92]:
# Check for duplicate contacts
# They have to match the same residue index, atom, and LIG atom
def remove_duplicate_contacts(contacts):
    unique_contacts = []  # List to store unique contacts
    duplicate_contacts = []  # List to store duplicate contacts
    seen_contacts = set()  # Set to store keys of unique contacts (for fast lookup)

    for contact in contacts:        
        # Extract contact information
        res1, res_name1, atom_name1, res2, res_name2, atom_name2, distance = contact

        # Create a unique key for the contact
        key = (res1, atom_name1, res2, atom_name2)  # Use both residues and atoms for uniqueness

        # Check if the key is already in the set
        if key in seen_contacts:
            duplicate_contacts.append(contact)  # Store duplicates
        else:
            seen_contacts.add(key)
            unique_contacts.append(contact)  # Add unique contact to list

    return unique_contacts, duplicate_contacts

# Remove duplicate contacts
unique_contacts_cluster0, duplicate_contacts_cluster0 = remove_duplicate_contacts(all_contacts_cluster0)
unique_contacts_cluster1, duplicate_contacts_cluster1 = remove_duplicate_contacts(all_contacts_cluster1)

# Output the results
print(f"Number of unique contacts in cluster 0: {len(unique_contacts_cluster0)}")
print(f"Number of duplicate contacts in cluster 0: {len(duplicate_contacts_cluster0)}")

Number of unique contacts in cluster 0: 144
Number of duplicate contacts in cluster 0: 247


In [93]:
# Convert numpy arrays to tuples for sorting
unique_contacts_cluster0 = [tuple(contact) for contact in unique_contacts_cluster0]
duplicate_contacts_cluster0 = [tuple(contact) for contact in duplicate_contacts_cluster0]

# Sort duplicate and unique clusters
unique_contacts_cluster0 = sorted(unique_contacts_cluster0)
duplicate_contacts_cluster0 = sorted(duplicate_contacts_cluster0)
"""
# Print info from duplicate and unique clusters
print("Duplicate contacts in cluster 0:")
for contact in duplicate_contacts_cluster0:
    print(contact)

print("Unique contacts in cluster 0:")
for contact in unique_contacts_cluster0:
    print(contact)
"""

'\n# Print info from duplicate and unique clusters\nprint("Duplicate contacts in cluster 0:")\nfor contact in duplicate_contacts_cluster0:\n    print(contact)\n\nprint("Unique contacts in cluster 0:")\nfor contact in unique_contacts_cluster0:\n    print(contact)\n'

In [94]:
# Open SwissDock model in ChimeraX
# type "combine #1.1 #1.2" in the command line, or however many other models there are
# Get rid of all other models and save
# save /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output/TrwD_atp_swissDock_combined.pdb relModel #2

# Load the combined SwissDock model
swissdock_combined = ("/Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output/input/TrwD_atp_swissDock_combined.pdb")

In [95]:
# Function to create separate dictionaries for atom name matches and mismatches
def create_atom_mapping(amber_structure, swissdock_poses):
    # Load the entire structure and select model 1
    amber_traj = md.load(amber_structure)
    swissdock_traj = md.load(swissdock_poses)

    # Filter for CA atoms in the protein (only residues that are part of the protein)
    protein_ca_indices_amber = amber_traj.top.select("name CA and protein")
    ligand_heavy_indices_amber = amber_traj.top.select("resname ATP and not element H")

    # Select the same indices for SwissDock structure
    protein_ca_indices_swissDock = swissdock_traj.top.select("name CA and protein")
    ligand_heavy_indices_swissDock = swissdock_traj.top.select("resname LIG and not element H")

    # Initialize dictionaries to store matches and mismatches
    match_mapping = {}
    mismatch_mapping = {}

    # Map protein CA atoms with residue name and atom name
    #print("\nProtein CA Atom Mapping:")
    for idx_amber, idx_swiss in zip(protein_ca_indices_amber, protein_ca_indices_swissDock):
        res_name_amber = amber_traj.top.atom(idx_amber).residue.name
        atom_name_amber = amber_traj.top.atom(idx_amber).name
        res_name_swiss = swissdock_traj.top.atom(idx_swiss).residue.name
        atom_name_swiss = swissdock_traj.top.atom(idx_swiss).name
        
        #print(f"Amber: {res_name_amber} {atom_name_amber}, SwissDock: {res_name_swiss} {atom_name_swiss}")  # Debugging line
        
        if res_name_amber == res_name_swiss and atom_name_amber == atom_name_swiss:  # If both match
            match_mapping[(res_name_amber, atom_name_amber)] = (res_name_swiss, atom_name_swiss)
        else:  # If either residue or atom names don't match
            mismatch_mapping[(res_name_amber, atom_name_amber)] = (res_name_swiss, atom_name_swiss)

    # Map ligand heavy atoms with residue name and atom name
    #print("\nLigand Heavy Atom Mapping:")
    for idx_amber, idx_swiss in zip(ligand_heavy_indices_amber, ligand_heavy_indices_swissDock):
        res_name_amber = amber_traj.top.atom(idx_amber).residue.name
        atom_name_amber = amber_traj.top.atom(idx_amber).name
        res_name_swiss = swissdock_traj.top.atom(idx_swiss).residue.name
        atom_name_swiss = swissdock_traj.top.atom(idx_swiss).name
        
        #print(f"Amber: {res_name_amber} {atom_name_amber}, SwissDock: {res_name_swiss} {atom_name_swiss}")  # Debugging line
        
        if res_name_amber == res_name_swiss and atom_name_amber == atom_name_swiss:  # If both match
            match_mapping[(res_name_amber, atom_name_amber)] = (res_name_swiss, atom_name_swiss)
        else:  # If either residue or atom names don't match
            mismatch_mapping[(res_name_amber, atom_name_amber)] = (res_name_swiss, atom_name_swiss)

    return match_mapping, mismatch_mapping

# SwissDock Cluster0
swissdock_cluster0 = "/Users/stuytschaevers/Desktop/Thesis/atp_mg2+/swissDock/combined/TrwD_atp_cluster_0.pdb"

# Create the atom mappings using only model 1 of SwissDock (for CA of protein and heavy atoms of ligand)
match_mapping, mismatch_mapping = create_atom_mapping(amber_minimized_structure, swissdock_cluster0)

# Print the match and mismatch mapping dictionaries
"""print("\nMatch Mapping:")
print(match_mapping)

print("\nMismatch Mapping:")
print(mismatch_mapping)
"""

'print("\nMatch Mapping:")\nprint(match_mapping)\n\nprint("\nMismatch Mapping:")\nprint(mismatch_mapping)\n'

In [96]:
# Function to get all ligand heavy atoms
def get_ligand_heavy_atoms(traj, lig_name):
    return traj.top.select(f"resname {lig_name} and not element H")

# Load the combined SwissDock model as an MDTraj trajectory
swissdock_combined_traj = md.load(swissdock_cluster0)

# Load the AMBER minimized structure as an MDTraj trajectory
amber_minimized_traj = md.load(amber_minimized_structure)

# Use function to get ligand heavy atoms for the combined SwissDock model
ligand_heavy_indices_swissDock = get_ligand_heavy_atoms(swissdock_combined_traj, "LIG")

# Use function to get ligand heavy atoms for the AMBER model
ligand_heavy_indices_amber = get_ligand_heavy_atoms(amber_minimized_traj, "ATP")

"""
print(f"Number of ligand heavy atoms in SwissDock: {len(ligand_heavy_indices_swissDock)}")
print(f"Number of ligand heavy atoms in AMBER: {len(ligand_heavy_indices_amber)}")

# Print all ligand heavy atoms
print("Ligand Heavy Atoms in SwissDock:")
for idx in ligand_heavy_indices_swissDock:
    print(swissdock_combined_traj.top.atom(idx))

print("\nLigand Heavy Atoms in AMBER:")
for idx in ligand_heavy_indices_amber:
    print(amber_minimized_traj.top.atom(idx))
"""

'\nprint(f"Number of ligand heavy atoms in SwissDock: {len(ligand_heavy_indices_swissDock)}")\nprint(f"Number of ligand heavy atoms in AMBER: {len(ligand_heavy_indices_amber)}")\n\n# Print all ligand heavy atoms\nprint("Ligand Heavy Atoms in SwissDock:")\nfor idx in ligand_heavy_indices_swissDock:\n    print(swissdock_combined_traj.top.atom(idx))\n\nprint("\nLigand Heavy Atoms in AMBER:")\nfor idx in ligand_heavy_indices_amber:\n    print(amber_minimized_traj.top.atom(idx))\n'

In [97]:
# NOTE: This block of code should be changed so you do not have to sort the lists


# Compare atom name of ligand heavy atoms between SwissDock and AMBER
ligand_heavy_swissDock = [swissdock_combined_traj.top.atom(idx).name for idx in ligand_heavy_indices_swissDock]
ligand_heavy_amber = [amber_minimized_traj.top.atom(idx).name for idx in ligand_heavy_indices_amber]

# Sort the lists for comparison
ligand_heavy_swissDock.sort()
ligand_heavy_amber.sort()

# Print the atom names for comparison
#print(ligand_heavy_swissDock)
#print(ligand_heavy_amber)

# List to store the differences in atom names
atom_name_differences = []
atom_name_same = []

# Compare atom names between SwissDock and AMBER
for name_swissDock, name_amber in zip(ligand_heavy_swissDock, ligand_heavy_amber):
    if name_swissDock != name_amber:
        atom_name_differences.append((name_swissDock, name_amber))
    else:
        atom_name_same.append((name_swissDock, name_amber))

# Print the differences in atom names
"""print("Atom Name Differences:")
for name_swissDock, name_amber in atom_name_differences:
    print(f"SwissDock: {name_swissDock}, AMBER: {name_amber}")

# Print the atom names that are the same
print("\nAtom Name Matches:")
for name_swissDock, name_amber in atom_name_same:
    print(f"SwissDock: {name_swissDock}, AMBER: {name_amber}")"""


'print("Atom Name Differences:")\nfor name_swissDock, name_amber in atom_name_differences:\n    print(f"SwissDock: {name_swissDock}, AMBER: {name_amber}")\n\n# Print the atom names that are the same\nprint("\nAtom Name Matches:")\nfor name_swissDock, name_amber in atom_name_same:\n    print(f"SwissDock: {name_swissDock}, AMBER: {name_amber}")'

In [98]:
# Function to get ligand heavy atom information
def get_ligand_heavy_atom_info(traj, lig_name):
    # Select ligand heavy atoms
    ligand_heavy_indices = traj.top.select(f"resname {lig_name} and not element H")
    
    # Dictionary to store residue number, atom name, and atom index
    atom_info = {}

    # Loop through each ligand heavy atom
    for idx in ligand_heavy_indices:
        atom = traj.top.atom(idx)
        res_num = atom.residue.index
        atom_name = atom.name
        
        # Store multiple atoms for the same residue
        if res_num not in atom_info:
            atom_info[res_num] = []
        atom_info[res_num].append((atom_name, idx))
    
    return atom_info

# Get ligand heavy atom information for SwissDock and AMBER
ligand_heavy_info_swissDock = get_ligand_heavy_atom_info(swissdock_combined_traj, "LIG")
ligand_heavy_info_amber = get_ligand_heavy_atom_info(amber_minimized_traj, "ATP")

# Print the ligand heavy atom information for SwissDock
"""print("Ligand Heavy Atom Information for SwissDock:")
for res_num, atoms in ligand_heavy_info_swissDock.items():
    for atom_name, atom_idx in atoms:
        print(f"Residue Number: {res_num}, Atom Name: {atom_name}, Atom Index: {atom_idx}")

# Print the ligand heavy atom information for AMBER
print("\nLigand Heavy Atom Information for AMBER:")
for res_num, atoms in ligand_heavy_info_amber.items():
    for atom_name, atom_idx in atoms:
        print(f"Residue Number: {res_num}, Atom Name: {atom_name}, Atom Index: {atom_idx}")"""

'print("Ligand Heavy Atom Information for SwissDock:")\nfor res_num, atoms in ligand_heavy_info_swissDock.items():\n    for atom_name, atom_idx in atoms:\n        print(f"Residue Number: {res_num}, Atom Name: {atom_name}, Atom Index: {atom_idx}")\n\n# Print the ligand heavy atom information for AMBER\nprint("\nLigand Heavy Atom Information for AMBER:")\nfor res_num, atoms in ligand_heavy_info_amber.items():\n    for atom_name, atom_idx in atoms:\n        print(f"Residue Number: {res_num}, Atom Name: {atom_name}, Atom Index: {atom_idx}")'

In [99]:
# Function to get all atom names from a dictionary
def get_all_atom_names(atom_info):
    all_atom_names = set()  # Use a set to store unique atom names
    
    # Loop through each residue number and atom name
    for res_num, atoms in atom_info.items():
        for atom_name, _ in atoms:
            all_atom_names.add(atom_name)
    
    return all_atom_names

# Get all atom names for SwissDock and AMBER
all_atom_names_swissDock = get_all_atom_names(ligand_heavy_info_swissDock)
all_atom_names_amber = get_all_atom_names(ligand_heavy_info_amber)
"""
# Print the unique atom names for SwissDock
print("Unique Atom Names for SwissDock:")
for atom_name in all_atom_names_swissDock:
    print(atom_name)

# Print the unique atom names for AMBER
print("\nUnique Atom Names for AMBER:")
for atom_name in all_atom_names_amber:
    print(atom_name)
"""


'\n# Print the unique atom names for SwissDock\nprint("Unique Atom Names for SwissDock:")\nfor atom_name in all_atom_names_swissDock:\n    print(atom_name)\n\n# Print the unique atom names for AMBER\nprint("\nUnique Atom Names for AMBER:")\nfor atom_name in all_atom_names_amber:\n    print(atom_name)\n'

In [100]:
# Function to print atom names to a text file
def write_atom_names_to_file(atom_names, filename):
    with open(filename, 'w') as f:
        for atom_name in atom_names:
            f.write(atom_name + "\n")

# Save the unique atom names to text files
write_atom_names_to_file(all_atom_names_swissDock, f"{output_folder}/unique_atom_names_swissDock.txt")
write_atom_names_to_file(all_atom_names_amber, f"{output_folder}/unique_atom_names_amber.txt")

In [101]:
# Funcation to get the similarities and differences in atom names
def compare_atom_names(atom_info_swissDock, atom_info_amber):
    # Initialize lists to store similarities and differences
    same_atom_names = []
    different_atom_names = []
    
    # Loop through each residue number and atom name
    for res_num, atoms_swissDock in atom_info_swissDock.items():
        atoms_amber = atom_info_amber.get(res_num, [])  # Get corresponding atoms in AMBER
        
        # Loop through each atom name and index in SwissDock
        for atom_name_swissDock, _ in atoms_swissDock:
            found = False  # Flag to check if the atom name is found in AMBER
            
            # Loop through each atom name and index in AMBER
            for atom_name_amber, _ in atoms_amber:
                if atom_name_swissDock == atom_name_amber:  # If the atom names match
                    same_atom_names.append((res_num, atom_name_swissDock))
                    found = True
                    break
            
            # If the atom name was not found in AMBER
            if not found:
                different_atom_names.append((res_num, atom_name_swissDock))
    
    return same_atom_names, different_atom_names

# Compare atom names between SwissDock and AMBER
same_atom_names, different_atom_names = compare_atom_names(ligand_heavy_info_swissDock, ligand_heavy_info_amber)
"""
# Print the atom names that are the same
print("Atom Names that Match:")
for res_num, atom_name in same_atom_names:
    print(f"Residue Number: {res_num}, Atom Name: {atom_name}")

# Print the atom names that are different
print("\nAtom Names that Differ:")
for res_num, atom_name in different_atom_names:
    print(f"Residue Number: {res_num}, Atom Name: {atom_name}")
"""

'\n# Print the atom names that are the same\nprint("Atom Names that Match:")\nfor res_num, atom_name in same_atom_names:\n    print(f"Residue Number: {res_num}, Atom Name: {atom_name}")\n\n# Print the atom names that are different\nprint("\nAtom Names that Differ:")\nfor res_num, atom_name in different_atom_names:\n    print(f"Residue Number: {res_num}, Atom Name: {atom_name}")\n'

In [102]:
# O5* in AMBER
# O5' in SwissDock

# Dictionary to change atom names
# SwissDock to AMBER
my_dict_diff = {"05'": "O5*"}

# All other atom names are the same

In [103]:
# Function get atom info from a given trajectory
def extract_atom_info(traj, atom_indices):
    """
    Extracts atom index, atom name, residue name, and residue number 
    for given atom indices in an mdtraj trajectory.

    Parameters:
        traj (mdtraj.Trajectory): The trajectory object.
        atom_indices (list): List of atom indices to extract.

    Returns:
        list: A list of tuples (atom_index, atom_name, residue_name, residue_number).
    """
    atom_info_list = []
    for idx in atom_indices:
        atom = traj.top.atom(idx)
        #atom_index = idx + 1  # Convert to 1-based indexing
        atom_index = idx + 2
        atom_name = atom.name
        residue_name = atom.residue.name
        residue_number = atom.residue.resSeq #+ 1  # Convert to 1-based indexing

        atom_info_list.append((atom_index, atom_name, residue_name, residue_number))

    return atom_info_list

# Load trajectories
amber_traj = md.load(amber_minimized_structure)
swissdock_traj = md.load(swissdock_cluster0)

# Extract for AMBER trajectory
amber_atom_info_list = extract_atom_info(amber_traj, ligand_heavy_indices_amber)

# Extract for SwissDock trajectory
swissdock_atom_info_list = extract_atom_info(swissdock_traj, ligand_heavy_indices_swissDock)

# Print the extracted lists
print("AMBER Atom Info List:")
print(amber_atom_info_list)

print("\nSwissDock Atom Info List:")
print(swissdock_atom_info_list)


AMBER Atom Info List:
[(2827, 'O1G', 'ATP', 359), (2828, 'PG', 'ATP', 359), (2829, 'O2G', 'ATP', 359), (2830, 'O3G', 'ATP', 359), (2831, 'O3B', 'ATP', 359), (2832, 'PB', 'ATP', 359), (2833, 'O1B', 'ATP', 359), (2834, 'O2B', 'ATP', 359), (2835, 'O3A', 'ATP', 359), (2836, 'PA', 'ATP', 359), (2837, 'O1A', 'ATP', 359), (2838, 'O2A', 'ATP', 359), (2839, 'O5*', 'ATP', 359), (2840, "C5'", 'ATP', 359), (2843, "C4'", 'ATP', 359), (2845, "C3'", 'ATP', 359), (2847, "O3'", 'ATP', 359), (2849, "C2'", 'ATP', 359), (2851, "O2'", 'ATP', 359), (2853, "O4'", 'ATP', 359), (2854, "C1'", 'ATP', 359), (2856, 'N9', 'ATP', 359), (2857, 'C8', 'ATP', 359), (2859, 'N7', 'ATP', 359), (2860, 'C5', 'ATP', 359), (2861, 'C6', 'ATP', 359), (2862, 'N6', 'ATP', 359), (2865, 'N1', 'ATP', 359), (2866, 'C2', 'ATP', 359), (2868, 'N3', 'ATP', 359), (2869, 'C4', 'ATP', 359)]

SwissDock Atom Info List:
[(2827, 'N9', 'LIG', 1), (2828, 'C8', 'LIG', 1), (2829, 'N7', 'LIG', 1), (2830, 'C5', 'LIG', 1), (2831, 'C6', 'LIG', 1), (2832

In [104]:
def create_mapping_from_files(swissDock_file, amber_file):
    """
    Reads atom names from the SwissDock and AMBER text files, compares the atom names, 
    and creates a mapping dictionary with matched atom names and manual mapping for "05'" to "O5*".
    
    Parameters:
        swissDock_file (str): Path to the text file containing atom names from SwissDock.
        amber_file (str): Path to the text file containing atom names from AMBER.
    
    Returns:
        dict: Dictionary of atom name mappings where keys are AMBER atom names and values are SwissDock atom names.
    """
    # Read the SwissDock atom names from the file
    with open(swissDock_file, 'r') as f:
        swissdock_atom_names = f.read().splitlines()
    
    # Read the AMBER atom names from the file
    with open(amber_file, 'r') as f:
        amber_atom_names = f.read().splitlines()
    
    # Initialize the mapping dictionary with the manual mapping
    atom_name_mapping_dict = {
        "05'": "O5*",
        # You can add other manual mappings here if needed
    }

    # Create a set of atom names from both lists for fast comparison
    swissdock_set = set(swissdock_atom_names)
    amber_set = set(amber_atom_names)
    
    # Find the matching atom names and add them to the dictionary
    for amber_name in amber_set:
        if amber_name in swissdock_set:
            atom_name_mapping_dict[amber_name] = amber_name
    
    return atom_name_mapping_dict

# Example usage
swissDock_file = f"{output_folder}/unique_atom_names_swissDock.txt"
amber_file = f"{output_folder}/unique_atom_names_amber.txt"

# Create the atom name mapping dictionary
atom_name_mapping_dict = create_mapping_from_files(swissDock_file, amber_file)

# Print the resulting atom name mapping dictionary
print("Atom Name Mapping Dictionary:")
print(atom_name_mapping_dict)


Atom Name Mapping Dictionary:
{"05'": 'O5*', 'C8': 'C8', 'O1G': 'O1G', 'C5': 'C5', 'O2G': 'O2G', 'N3': 'N3', "O4'": "O4'", 'PB': 'PB', 'N1': 'N1', 'C4': 'C4', "C2'": "C2'", 'O3G': 'O3G', 'N6': 'N6', "C1'": "C1'", 'O3B': 'O3B', 'N9': 'N9', 'O2B': 'O2B', 'N7': 'N7', "O3'": "O3'", 'O3A': 'O3A', 'PA': 'PA', 'C2': 'C2', "C3'": "C3'", "O2'": "O2'", 'PG': 'PG', 'O1B': 'O1B', 'O1A': 'O1A', 'O2A': 'O2A', 'C6': 'C6', "C4'": "C4'", "C5'": "C5'"}


In [105]:
def match_atom_names_to_dict(amber_atom_info_list, swissdock_atom_info_list, atom_name_dict):
    """
    Matches atom names between AMBER and SwissDock atom info lists.
    Creates a dictionary where keys are tuples of SwissDock atom information
    and values are tuples of corresponding AMBER atom information.
    If a match is found, it updates the atom name using the dictionary, including manual mappings.
    
    Parameters:
        amber_atom_info_list (list): List of tuples (atom_index, atom_name, residue_name, residue_number) for AMBER.
        swissdock_atom_info_list (list): List of tuples (atom_index, atom_name, residue_name, residue_number) for SwissDock.
        atom_name_dict (dict): Dictionary mapping original atom names to new atom names.
    
    Returns:
        dict: Dictionary with tuples (atom_index, atom_name, residue_name, residue_number) as keys and values.
    """
    matched_atoms_dict = {}

    # Iterate through both lists and create the mapping
    for amber_info in amber_atom_info_list:
        amber_index, amber_name, amber_residue, amber_res_num = amber_info

        # Apply manual mapping if available
        updated_amber_name = atom_name_dict.get(amber_name, amber_name)

        #print(f"Looking for AMBER atom: {amber_name} (updated: {updated_amber_name})")

        # Find the matching SwissDock info
        for swissdock_info in swissdock_atom_info_list:
            swissdock_index, swissdock_name, swissdock_residue, swissdock_res_num = swissdock_info

            # Apply manual mapping to SwissDock atom names if necessary (for "05'" -> "O5*")
            updated_swissdock_name = atom_name_dict.get(swissdock_name, swissdock_name)
            if updated_swissdock_name == "05'":
                updated_swissdock_name = "O5*"

            #print(f"Checking SwissDock atom: {updated_swissdock_name} with residue {swissdock_residue} {swissdock_res_num}")

            # Now check if atom names and residues match (including updated names)
            if (updated_swissdock_name == updated_amber_name):
                #print(f"Match found! SwissDock: {swissdock_info} -> AMBER: {amber_info}")
                # If there's a match, store it
                matched_atoms_dict[(swissdock_index, updated_swissdock_name, swissdock_residue, swissdock_res_num)] = (
                    amber_index, updated_amber_name, amber_residue, amber_res_num
                )
                break  # Exit inner loop once a match is found

    if not matched_atoms_dict:
        print("No matches were found.")

    return matched_atoms_dict

# Example of calling the function
matched_atoms_dict = match_atom_names_to_dict(amber_atom_info_list, swissdock_atom_info_list, atom_name_mapping_dict)

# Print the matched atoms dictionary if there are matches
if matched_atoms_dict:
    for swissdock_info, amber_info in matched_atoms_dict.items():
        print(f"SwissDock: {swissdock_info} -> AMBER: {amber_info}")
else:
    print("No matching atoms found.")


SwissDock: (2836, 'O1G', 'LIG', 1) -> AMBER: (2827, 'O1G', 'ATP', 359)
SwissDock: (2837, 'PG', 'LIG', 1) -> AMBER: (2828, 'PG', 'ATP', 359)
SwissDock: (2838, 'O2G', 'LIG', 1) -> AMBER: (2829, 'O2G', 'ATP', 359)
SwissDock: (2839, 'O3G', 'LIG', 1) -> AMBER: (2830, 'O3G', 'ATP', 359)
SwissDock: (2840, 'O3B', 'LIG', 1) -> AMBER: (2831, 'O3B', 'ATP', 359)
SwissDock: (2841, 'PB', 'LIG', 1) -> AMBER: (2832, 'PB', 'ATP', 359)
SwissDock: (2842, 'O1B', 'LIG', 1) -> AMBER: (2833, 'O1B', 'ATP', 359)
SwissDock: (2843, 'O2B', 'LIG', 1) -> AMBER: (2834, 'O2B', 'ATP', 359)
SwissDock: (2844, 'O3A', 'LIG', 1) -> AMBER: (2835, 'O3A', 'ATP', 359)
SwissDock: (2845, 'PA', 'LIG', 1) -> AMBER: (2836, 'PA', 'ATP', 359)
SwissDock: (2846, 'O1A', 'LIG', 1) -> AMBER: (2837, 'O1A', 'ATP', 359)
SwissDock: (2847, 'O2A', 'LIG', 1) -> AMBER: (2838, 'O2A', 'ATP', 359)
SwissDock: (2849, "C5'", 'LIG', 1) -> AMBER: (2840, "C5'", 'ATP', 359)
SwissDock: (2850, "C4'", 'LIG', 1) -> AMBER: (2843, "C4'", 'ATP', 359)
SwissDock: (

In [106]:
# Add key to the dictionary
# Key: (residue number, atom name)
# Value: (residue number, atom name)

key1 = (2848, "O5*", 'LIG', 359)
value1 = (5694, "O5'", 'ATP', 359)

key2 = (2848, "O5'", 'LIG', 359)
value2 = (5694, "O5*", 'ATP', 359)

# Add the key-value pair to the dictionary
matched_atoms_dict[key1] = value1
matched_atoms_dict[key2] = value2

for swissdock_info, amber_info in matched_atoms_dict.items():
    print(f"SwissDock: {swissdock_info} -> AMBER: {amber_info}")

SwissDock: (2836, 'O1G', 'LIG', 1) -> AMBER: (2827, 'O1G', 'ATP', 359)
SwissDock: (2837, 'PG', 'LIG', 1) -> AMBER: (2828, 'PG', 'ATP', 359)
SwissDock: (2838, 'O2G', 'LIG', 1) -> AMBER: (2829, 'O2G', 'ATP', 359)
SwissDock: (2839, 'O3G', 'LIG', 1) -> AMBER: (2830, 'O3G', 'ATP', 359)
SwissDock: (2840, 'O3B', 'LIG', 1) -> AMBER: (2831, 'O3B', 'ATP', 359)
SwissDock: (2841, 'PB', 'LIG', 1) -> AMBER: (2832, 'PB', 'ATP', 359)
SwissDock: (2842, 'O1B', 'LIG', 1) -> AMBER: (2833, 'O1B', 'ATP', 359)
SwissDock: (2843, 'O2B', 'LIG', 1) -> AMBER: (2834, 'O2B', 'ATP', 359)
SwissDock: (2844, 'O3A', 'LIG', 1) -> AMBER: (2835, 'O3A', 'ATP', 359)
SwissDock: (2845, 'PA', 'LIG', 1) -> AMBER: (2836, 'PA', 'ATP', 359)
SwissDock: (2846, 'O1A', 'LIG', 1) -> AMBER: (2837, 'O1A', 'ATP', 359)
SwissDock: (2847, 'O2A', 'LIG', 1) -> AMBER: (2838, 'O2A', 'ATP', 359)
SwissDock: (2849, "C5'", 'LIG', 1) -> AMBER: (2840, "C5'", 'ATP', 359)
SwissDock: (2850, "C4'", 'LIG', 1) -> AMBER: (2843, "C4'", 'ATP', 359)
SwissDock: (

In [107]:
# Function to count how many atom names are in a text file
def count_atom_names_in_file(filename):
    with open(filename, 'r') as f:
        atom_names = f.read().splitlines()
    return len(atom_names)

# Count the number of atom names in the SwissDock and AMBER files
num_atom_names_swissDock = count_atom_names_in_file(swissDock_file)
num_atom_names_amber = count_atom_names_in_file(amber_file)

# Print the number of atom names in each file
print(f"Number of atom names in SwissDock file: {num_atom_names_swissDock}")
print(f"Number of atom names in AMBER file: {num_atom_names_amber}")

Number of atom names in SwissDock file: 31
Number of atom names in AMBER file: 31


In [108]:
# Get the length of the dictionary 
print(len(matched_atoms_dict))

if len(matched_atoms_dict) == num_atom_names_swissDock:
    print("All atom names have been matched.")

32


In [109]:
import os
import re

def match_atom_names_in_files(output_dir, matched_atoms_dict):
    all_updated_lines = []
    contact_counter = 0

    print(f"Checking files in directory: {output_dir}")
    
    # Define the pattern for files like contacts_cluster1.pb, contacts_clusterXX.pb
    pattern = re.compile(r"^contacts_cluster\d+\.pb$")

    for file_name in os.listdir(output_dir):
        # Use regex pattern to match files like contacts_cluster1.pb or contacts_clusterXX.pb
        if pattern.match(file_name):
            file_path = os.path.join(output_dir, file_name)
            print(f"Processing file: {file_name}")

            with open(file_path, 'r') as file:
                lines = file.readlines()

            if not lines:
                print(f"Warning: No lines found in {file_name}")
            
            for line in lines:
                contact_counter += 1
                
                # Skip lines that don't match the format of atom contacts
                if line.startswith("#"):
                    print(f"Skipping header line: {line.strip()}")
                    continue

                # If the line has atom contact information, process it
                parts = line.split()
                if len(parts) < 3:
                    print(f"Skipping line due to insufficient parts: {line.strip()}")
                    continue

                swissdock_info = parts[0]
                amber_info = parts[1]
                distance = parts[2]

                atom_name1 = swissdock_info.split('@')[-1]
                atom_name2 = amber_info.split('@')[-1]
                print(f"Extracted atom names: {atom_name1}, {atom_name2} with distance: {distance}")

                # Check if the atom name is O5* or O5'
                if atom_name2 in ["O5*", "O5'"]:
                    print(f"Found atom {atom_name2} in line: {line.strip()}")

                # Loop through matched_atoms_dict to find a match
                for swissdock_info_key, amber_info_value in matched_atoms_dict.items():
                    swissdock_atom_name = swissdock_info_key[1]
                    amber_atom_name = amber_info_value[1]

                    #print(f"Matching atom: {atom_name2} with {amber_atom_name}")  # Debugging match

                    if atom_name2.strip() == amber_atom_name.strip():  # Using strip for matching
                        print(f"Match found: {atom_name2} in matched_atoms_dict")
                        res_name1 = swissdock_info.split(':')[1].split('@')[0]
                        lig_res_name = amber_info_value[2]
                        
                        new_line = f"/A:{res_name1}@{atom_name1} /B:{lig_res_name}@{atom_name2} {distance}\n"
                        print(f"New formatted line: {new_line.strip()}")
                        all_updated_lines.append(new_line)
                        break  # Found the match, no need to check further

            # Add a blank line between contact files
            all_updated_lines.append("\n")
            print(f"Processed {contact_counter} contacts in file: {file_name}")
            contact_counter = 0

    return all_updated_lines


updated_lines = match_atom_names_in_files(output_folder, matched_atoms_dict)

#print(updated_lines)

Checking files in directory: /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output
Processing file: contacts_cluster0.pb
Extracted atom names: CA, C6 with distance: 0.487275630235672
Match found: C6 in matched_atoms_dict
New formatted line: /A:13@CA /B:ATP@C6 0.487275630235672
Extracted atom names: CA, N6 with distance: 0.3990568220615387
Match found: N6 in matched_atoms_dict
New formatted line: /A:13@CA /B:ATP@N6 0.3990568220615387
Extracted atom names: CA, C6 with distance: 0.43232545256614685
Match found: C6 in matched_atoms_dict
New formatted line: /A:14@CA /B:ATP@C6 0.43232545256614685
Extracted atom names: CA, N1 with distance: 0.40078720450401306
Match found: N1 in matched_atoms_dict
New formatted line: /A:14@CA /B:ATP@N1 0.40078720450401306
Extracted atom names: CA, C2 with distance: 0.48034942150115967
Match found: C2 in matched_atoms_dict
New formatted line: /A:14@CA /B:ATP@C2 0.48034942150115967
Extracted atom names: CA, N6 with distance: 0.41062653064727783
Match fou

In [110]:
def write_updated_contacts(output_dir, updated_lines):
    """
    Writes the updated contact lines to a final contacts file.
    
    Parameters:
        output_dir (str): The directory where the final contacts file will be saved.
        updated_lines (list): List of updated contact lines to write.
    """
    # Define the final output file path
    final_output_file_path = os.path.join(output_dir, "final_contacts_updated.pb")

    # If the file already exists, delete it
    if os.path.exists(final_output_file_path):
        os.remove(final_output_file_path)

    # Save the final contacts file with all the updated lines and a blank line at the end
    with open(final_output_file_path, 'w') as final_output_file:
        final_output_file.writelines(updated_lines)
    
    print(f"Final updated contacts saved to {final_output_file_path}")

# Write the updated contacts to the file
write_updated_contacts(output_folder, updated_lines)

Final updated contacts saved to /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output/final_contacts_updated.pb


In [111]:
# Function to create .dat file exactly like the .pb file
def create_dat_file_from_pb(pb_file, dat_file):
    """
    Creates a .dat file from a .pb file with the same contact information.
    
    Parameters:
        pb_file (str): Path to the .pb file.
        dat_file (str): Path to the .dat file to create.
    """
    # Read the .pb file
    with open(pb_file, 'r') as pb:
        lines = pb.readlines()
    
    # Write the same lines to the .dat file
    with open(dat_file, 'w') as dat:
        dat.writelines(lines)
    
    print(f".dat file created from {pb_file} and saved to {dat_file}")

# Create a .dat file from the .pb file
pb_file = f"{output_folder}/final_contacts_updated.pb"
dat_file = f"{output_folder}/final_contacts_updated.dat"
create_dat_file_from_pb(pb_file, dat_file)

.dat file created from /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output/final_contacts_updated.pb and saved to /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output/final_contacts_updated.dat


In [112]:
# Function to ensure there is only one blank line at the end of the file
def remove_extra_blank_lines(file_path):
    """
    Removes extra blank lines at the end of a file.
    
    Parameters:
        file_path (str): Path to the file to check.
    """
    # Read the file
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Remove extra blank lines at the end
    while lines and lines[-1].strip() == "":
        lines.pop()
    
    # Write the updated lines back to the file
    with open(file_path, 'w') as file:
        file.writelines(lines)
    
    print(f"Extra blank lines removed from {file_path}")

# Remove extra blank lines from the final .pb file
remove_extra_blank_lines(pb_file)
remove_extra_blank_lines(dat_file)

Extra blank lines removed from /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output/final_contacts_updated.pb
Extra blank lines removed from /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output/final_contacts_updated.dat


In [113]:
def get_max_distance_from_file(file_path):
    """
    Reads a contacts file and returns the maximum distance value.
    
    Parameters:
        file_path (str): Path to the contacts file.
    
    Returns:
        float: The maximum distance value found in the file.
    """
    max_distance = 0.0

    with open(file_path, 'r') as file:
        for line in file:
            parts = line.split()
            if len(parts) >= 3:  # Ensure there is a distance value in the line
                try:
                    distance = float(parts[2])
                    max_distance = max(max_distance, distance)
                except ValueError:
                    continue  # Skip lines where the third column isn't a valid float
    
    return max_distance

# Example usage
dat_file = "/Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output/final_contacts_updated.dat"  # Replace with actual file path
max_distance = get_max_distance_from_file(dat_file)
print(f"Maximum distance in the final contacts file: {max_distance}")

# Function to write max distance in a file
def write_max_distance_to_file(max_distance, output_dir):
    """
    Writes the maximum distance value to a text file.
    
    Parameters:
        max_distance (float): The maximum distance value to write.
        output_dir (str): The directory where the text file will be saved.
    """
    # Define the output file path
    output_file_path = os.path.join(output_dir, "max_distance.txt")

    # Write the maximum distance to the file
    with open(output_file_path, 'w') as file:
        file.write(str(max_distance))
    
    print(f"Maximum distance written to {output_file_path}")

# Example usage
write_max_distance_to_file(max_distance, output_folder)


Maximum distance in the final contacts file: 0.49950671195983887
Maximum distance written to /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output/max_distance.txt


In [114]:
# Funtion to open directory and parse through .pb files
# Replace "ATP" with 359

def edit_contacts(directory):
    """
    Edits the contact files in the given directory to replace "ATP" with "359".
    
    Parameters:
        directory (str): The directory containing the contact files.
    """
    for file_name in os.listdir(directory):
        if not file_name.endswith(".pb"):  # Process only .pb files
            continue

        file_path = os.path.join(directory, file_name)
        print(f"Processing file: {file_name}")

        with open(file_path, 'r', encoding='latin1') as file:
            lines = file.readlines()

        if not lines:
            print(f"Warning: No lines found in {file_name}")
            continue
        
        updated_lines = []
        for line in lines:
            if line.startswith("#"):
                print(f"Skipping header line: {line.strip()}")
                updated_lines.append(line)
                continue

            parts = line.split()
            if len(parts) < 3:
                print(f"Skipping line due to insufficient parts: {line.strip()}")
                updated_lines.append(line)
                continue

            updated_line = line.replace("ATP", "359")
            updated_lines.append(updated_line)

        # Write back the modified content
        with open(file_path, 'w', encoding='latin1') as file:
            file.writelines(updated_lines)

        print(f"Updated contacts in file: {file_name}")

# Example usage
edit_contacts(output_folder)


Processing file: contacts_cluster0.pb
Updated contacts in file: contacts_cluster0.pb
Processing file: contacts_cluster1.pb
Updated contacts in file: contacts_cluster1.pb
Processing file: final_contacts_updated.pb
Skipping line due to insufficient parts: 
Updated contacts in file: final_contacts_updated.pb
Processing file: contacts_cluster1_visualization.pb
Skipping header line: #1.1/A:339@CA #1.1/B:LIG@N6
Skipping header line: #1.1/A:340@CA #1.1/B:LIG@N7
Skipping header line: #1.1/A:162@CA #1.1/B:LIG@C8
Skipping header line: #1.2/A:162@CA #1.2/B:LIG@N7
Skipping header line: #1.2/A:163@CA #1.2/B:LIG@C8
Skipping header line: #1.2/A:163@CA #1.2/B:LIG@N7
Skipping header line: #1.2/A:179@CA #1.2/B:LIG@O5*
Skipping header line: #1.2/A:179@CA #1.2/B:LIG@C5'
Skipping header line: #1.2/A:180@CA #1.2/B:LIG@N9
Skipping header line: #1.2/A:180@CA #1.2/B:LIG@C8
Skipping header line: #1.2/A:180@CA #1.2/B:LIG@N7
Skipping header line: #1.2/A:180@CA #1.2/B:LIG@O5*
Skipping header line: #1.2/A:180@CA #1

In [115]:
def reformat_restraints(matched_atoms_dict, input_file, output_file, output_folder, traj):
    all_updated_lines = []
    contact_counter = 0

    print(f"Checking files in directory: {output_folder}")
    
    # Define the pattern for files like contacts_cluster1.pb, contacts_clusterXX.pb
    pattern = re.compile(r"^contacts_cluster\d+\.pb$")

    for file_name in os.listdir(output_folder):
        if pattern.match(file_name):
            file_path = os.path.join(output_folder, file_name)
            print(f"Processing file: {file_name}")

            with open(file_path, 'r') as file:
                lines = file.readlines()

            if not lines:
                print(f"Warning: No lines found in {file_name}")
            
            for line in lines:
                contact_counter += 1

                if line.startswith("#"):
                    print(f"Skipping header line: {line.strip()}")
                    continue

                # If the line has atom contact information, process it
                parts = line.split()
                if len(parts) < 3:
                    print(f"Skipping line due to insufficient parts: {line.strip()}")
                    continue

                swissdock_info = parts[0]
                amber_info = parts[1]
                distance = parts[2]

                atom_name1 = swissdock_info.split('@')[-1]
                atom_name2 = amber_info.split('@')[-1]
                print(f"Extracted atom names: {atom_name1}, {atom_name2} with distance: {distance}")

                if atom_name2 in ["O5*", "O5'"]:
                    print(f"Found atom {atom_name2} in line: {line.strip()}")

                # Loop through matched_atoms_dict to find a match
                for swissdock_info_key, amber_info_value in matched_atoms_dict.items():
                    swissdock_atom_name = swissdock_info_key[1]
                    amber_atom_name = amber_info_value[1]

                    if atom_name2.strip() == amber_atom_name.strip():  # Using strip for matching
                        print(f"Match found: {atom_name2} in matched_atoms_dict")
                        
                        # Extract residue number from swissdock_info
                        res_num1 = int(swissdock_info.split(':')[1].split('@')[0])
                        
                        # Residue number from amber_info_value (might be used as res_num2)
                        lig_res_name = amber_info_value[2]

                        # Get protein residue name from trajectory
                        prot_res_name = traj.topology.atom(res_num1).residue.name

                        # Define residue number for ligand
                        res_num2 = 359
                        
                        # Format the new line with the appropriate residue number and atom names
                        new_line = f"{res_num1} {prot_res_name} {atom_name1} {res_num2} {lig_res_name} {atom_name2} {distance}\n"
                        print(f"New formatted line: {new_line.strip()}")
                        all_updated_lines.append(new_line)
                        break  # Found the match, no need to check further

            # Add a blank line between contact files
            all_updated_lines.append("\n")
            print(f"Processed {contact_counter} contacts in file: {file_name}")
            contact_counter = 0

    return all_updated_lines


# Example usage:
input_file = "/Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output/final_contacts_updated.pb"
output_file = "/Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output/final_contacts_updated.txt"

meld_updated_lines = reformat_restraints(matched_atoms_dict, input_file, output_file, output_folder, amber_traj)

# Write the updated lines to the output file
with open(output_file, 'w') as f:
    f.writelines(meld_updated_lines)


Checking files in directory: /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output
Processing file: contacts_cluster0.pb
Extracted atom names: CA, C6 with distance: 0.487275630235672
Match found: C6 in matched_atoms_dict
New formatted line: 13 SER CA 359 ATP C6 0.487275630235672
Extracted atom names: CA, N6 with distance: 0.3990568220615387
Match found: N6 in matched_atoms_dict
New formatted line: 13 SER CA 359 ATP N6 0.3990568220615387
Extracted atom names: CA, C6 with distance: 0.43232545256614685
Match found: C6 in matched_atoms_dict
New formatted line: 14 THR CA 359 ATP C6 0.43232545256614685
Extracted atom names: CA, N1 with distance: 0.40078720450401306
Match found: N1 in matched_atoms_dict
New formatted line: 14 THR CA 359 ATP N1 0.40078720450401306
Extracted atom names: CA, C2 with distance: 0.48034942150115967
Match found: C2 in matched_atoms_dict
New formatted line: 14 THR CA 359 ATP C2 0.48034942150115967
Extracted atom names: CA, N6 with distance: 0.41062653064727783

In [116]:
import sklearn

print(sklearn.__version__)


1.5.1


In [117]:
# Folders open
# /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/TrwD_atp_min.pdb
# /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/cluster/output/final_contacts_updated.pb
# /Users/stuytschaevers/Desktop/Thesis/atp_mg2+/swissDock/combined/TrwD_atp_cluster_0.pdb

In [118]:
"""def match_atom_names_in_files(output_dir, matched_atoms_dict):
    Processes .pb files, matches atom names with the given dictionary, and returns updated contact lines with distances.

    Parameters:
        output_dir (str): The directory where the .pb files are located.
        matched_atoms_dict (dict): Dictionary mapping SwissDock atom info to AMBER atom info.

    Returns:
        List of updated lines with the matched atom information and distances.
    all_updated_lines = []
    contact_counter = 0  # Counter for the number of contacts processed

    # Loop through all .pb files in the given output directory
    for file_name in os.listdir(output_dir):
        if file_name.endswith(".pb"):
            file_path = os.path.join(output_dir, file_name)
            
            with open(file_path, 'r') as file:
                lines = file.readlines()

            for line in lines:
                contact_counter += 1  # Increment the contact counter
                # Skip lines that don't match the format for atoms
                if line.startswith("#"):
                    # Extract atom info from the line
                    # Example format: #1.1/A:12@CA #1.1/B:LIG@C6 0.487275630235672
                    parts = line.split()
                    print(f"Processing line: {line.strip()}")
                    print(f"Extracted parts: {parts}")

                    if len(parts) < 3:
                        print(f"Skipping line due to insufficient parts: {line.strip()}")
                        continue

                    swissdock_info = parts[0]  # SwissDock atom info (e.g., #1.1/A:12@CA)
                    amber_info = parts[1]  # AMBER atom info (e.g., #1.1/B:LIG@C6)
                    distance = parts[2]  # Distance value (e.g., 0.487275630235672)

                    # Extract atom names (e.g., 'CA' and 'C6') from the contact string
                    atom_name1 = swissdock_info.split('@')[-1]  # Extract atom name from SwissDock info
                    atom_name2 = amber_info.split('@')[-1]  # Extract atom name from AMBER info

                    print(f"Extracted atom names: {atom_name1}, {atom_name2} with distance: {distance}")

                    # Check if the atom name is O5* or O5'
                    if atom_name2 in ["O5*", "O5'"]:
                        print(f"Found atom {atom_name2} in line: {line.strip()}")

                    # Loop through the matched_atoms_dict to find if atom_name2 exists in AMBER side
                    for swissdock_info_key, amber_info_value in matched_atoms_dict.items():
                        swissdock_atom_name = swissdock_info_key[1]  # SwissDock atom name from the key
                        amber_atom_name = amber_info_value[1]  # AMBER atom name from the value

                        # Check if the AMBER atom name matches the one from the line (atom_name2)
                        if atom_name2 == amber_atom_name:
                            print(f"Match found for {atom_name2} in matched_atoms_dict")

                            # Get the corresponding SwissDock and AMBER residue names
                            res1 = swissdock_info_key[2]  # SwissDock residue name
                            lig_res_name = amber_info_value[2]  # AMBER residue name
                            
                            # Extract the residue number from SwissDock info
                            protein_residue = swissdock_info.split(":")[1].split("@")[0]  

                            # Format the new line with distance
                            new_line = f"/A:{protein_residue}@{atom_name1} /B:{lig_res_name}@{atom_name2} {distance}\n"

                            print(f"New formatted line: {new_line.strip()}")
                            all_updated_lines.append(new_line)
                            break  # Found the match, no need to check further
                    else:
                        print(f"No match found for {atom_name2} in matched_atoms_dict")

            # Add a blank line between contact files
            all_updated_lines.append("\n")
            print(f"Processed {contact_counter} contacts in file: {file_name}")
            contact_counter = 0  # Reset the contact counter for the next file

    return all_updated_lines

# Example call
updated_lines = match_atom_names_in_files(output_folder, matched_atoms_dict)

# Uncomment to print the updated lines

print("Updated Lines:")
for line in updated_lines:
    print(line)

"""

'def match_atom_names_in_files(output_dir, matched_atoms_dict):\n    Processes .pb files, matches atom names with the given dictionary, and returns updated contact lines with distances.\n\n    Parameters:\n        output_dir (str): The directory where the .pb files are located.\n        matched_atoms_dict (dict): Dictionary mapping SwissDock atom info to AMBER atom info.\n\n    Returns:\n        List of updated lines with the matched atom information and distances.\n    all_updated_lines = []\n    contact_counter = 0  # Counter for the number of contacts processed\n\n    # Loop through all .pb files in the given output directory\n    for file_name in os.listdir(output_dir):\n        if file_name.endswith(".pb"):\n            file_path = os.path.join(output_dir, file_name)\n            \n            with open(file_path, \'r\') as file:\n                lines = file.readlines()\n\n            for line in lines:\n                contact_counter += 1  # Increment the contact counter\n     

In [119]:
# Old way of matching atom names in files without the distance

"""
def match_atom_names_in_files(output_dir, matched_atoms_dict):
    
    Processes .pb files, matches atom names with the given dictionary, and returns updated contact lines.
    
    Parameters:
        output_dir (str): The directory where the .pb files are located.
        matched_atoms_dict (dict): Dictionary mapping SwissDock atom info to AMBER atom info.
    
    Returns:
        List of updated lines with the matched atom information.
    
    all_updated_lines = []

    # Loop through all .pb files in the given output directory
    for file_name in os.listdir(output_dir):
        if file_name.endswith(".pb"):
            file_path = os.path.join(output_dir, file_name)
            
            with open(file_path, 'r') as file:
                lines = file.readlines()

            for line in lines:
                # Skip lines that don't match the format for atoms
                if line.startswith("#"):
                    # Extract atom info from the line
                    # Format example: #1.1/A:178@CA #1.1/B:LIG@O1A
                    parts = line.split()
                    print(f"Processing line: {line.strip()}")
                    print(f"Extracted parts: {parts}")

                    if len(parts) < 2:
                        print(f"Skipping line due to insufficient parts: {line.strip()}")
                        continue

                    swissdock_info = parts[0]  # First part corresponds to the SwissDock information
                    amber_info = parts[1]  # Second part corresponds to the AMBER information

                    # Extract atom names (e.g., 'CA' and 'O1A') from the contact string
                    atom_name1 = swissdock_info.split('@')[-1]  # Extract atom name from SwissDock info (e.g., 'CA')
                    atom_name2 = amber_info.split('@')[-1]  # Extract atom name from AMBER info (e.g., 'O1A')

                    print(f"Extracted atom names: {atom_name1}, {atom_name2}")

                    # Check if the atom name is O5* or O5'
                    if atom_name2 == "O5*" or atom_name2 == "O5'":
                        print(f"Found atom {atom_name2} in line: {line.strip()}")

                    # Loop through the matched_atoms_dict to find if atom_name2 exists in AMBER side
                    for swissdock_info_key, amber_info_value in matched_atoms_dict.items():
                        swissdock_atom_name = swissdock_info_key[1]  # SwissDock atom name from the key
                        amber_atom_name = amber_info_value[1]  # AMBER atom name from the value

                        #print(f"Checking against dict: SwissDock atom name: {swissdock_atom_name}, AMBER atom name: {amber_atom_name}")

                        # Check if the AMBER atom name matches the one from the line (atom_name2)
                        if atom_name2 == amber_atom_name:
                            print(f"Match found for {atom_name2} in matched_atoms_dict")

                            # Get the corresponding SwissDock and AMBER residue names
                            res1 = swissdock_info_key[2]  # SwissDock residue name
                            lig_res_name = amber_info_value[2]  # AMBER residue name
                            
                            # Format the new line
                            # Replace "/?" with actual protein and ligand residue names
                            # Assume that the residue number is extracted from the SwissDock info (in this case '12')
                            protein_residue = swissdock_info.split(":")[1].split("@")[0]  # Extract the residue number
                            new_line = f"/?:{protein_residue}@{atom_name1} /?:{lig_res_name}@{atom_name2} {distance} \n"

                            print(f"New formatted line: {new_line.strip()}")
                            all_updated_lines.append(new_line)
                            break  # Found the match, no need to check further
                    else:
                        print(f"No match found for {atom_name2} in matched_atoms_dict")

            # Add a blank line between contact files
            all_updated_lines.append("\n")

    return all_updated_lines

updated_lines = match_atom_names_in_files(output_folder, matched_atoms_dict)

# Print the updated lines
print("Updated Lines:")
for line in updated_lines:
    print(line)
"""

'\ndef match_atom_names_in_files(output_dir, matched_atoms_dict):\n    \n    Processes .pb files, matches atom names with the given dictionary, and returns updated contact lines.\n    \n    Parameters:\n        output_dir (str): The directory where the .pb files are located.\n        matched_atoms_dict (dict): Dictionary mapping SwissDock atom info to AMBER atom info.\n    \n    Returns:\n        List of updated lines with the matched atom information.\n    \n    all_updated_lines = []\n\n    # Loop through all .pb files in the given output directory\n    for file_name in os.listdir(output_dir):\n        if file_name.endswith(".pb"):\n            file_path = os.path.join(output_dir, file_name)\n            \n            with open(file_path, \'r\') as file:\n                lines = file.readlines()\n\n            for line in lines:\n                # Skip lines that don\'t match the format for atoms\n                if line.startswith("#"):\n                    # Extract atom info from 

In [120]:
"""import mdtraj as md

# Function to create a dictionary of atom name mappings (only for CA of protein and heavy atoms of the ligand)
def create_atom_mapping(amber_structure, swissdock_poses):
    print("Loading Amber and SwissDock structures...")
    
    # Load the entire structure (including all models) for both Amber and SwissDock poses
    amber_traj = md.load(amber_structure)
    swissdock_traj = md.load(swissdock_poses)
    
    print(f"Amber structure loaded with {amber_traj.n_atoms} atoms.")
    print(f"SwissDock structure loaded with {swissdock_traj.n_atoms} atoms.")
    
    # Filter for CA atoms in the protein (only residues that are part of the protein, exclude water)
    print("Filtering CA atoms from the Amber structure...")
    amber_atoms = [(atom.residue.name, atom.name) for atom in amber_traj.topology.atoms
                   if (atom.residue.name != 'HOH' and atom.name == 'CA')]  # Exclude water and get CA atoms
    print(f"Found {len(amber_atoms)} CA atoms in the Amber structure.")
    
    # Filter for heavy atoms in the ligand (excluding hydrogen atoms)
    print("Filtering heavy atoms (excluding hydrogens) from the SwissDock structure...")
    swissdock_atoms = [(atom.residue.name, atom.name) for atom in swissdock_traj.topology.atoms
                       if atom.element.symbol != 'H']  # Exclude hydrogens (heavy atoms only)
    print(f"Found {len(swissdock_atoms)} heavy atoms in the SwissDock structure.")
    
    # Create atom mapping only for mismatched atom names between the two structures
    print("Creating atom mapping...")
    atom_mapping = {}
    for (res1, atom_name1), (res2, atom_name2) in zip(amber_atoms, swissdock_atoms):
        if atom_name1 != atom_name2:
            atom_mapping[atom_name1] = atom_name2
            print(f"Mapping: {atom_name1} -> {atom_name2}")
    
    print(f"Total mappings created: {len(atom_mapping)}")
    
    return atom_mapping

# Create the atom mapping dictionary using the pre-defined paths for the structures
atom_mapping = create_atom_mapping(amber_minimized_structure, swissdock_one)

# Print the atom mapping dictionary
print("Atom Mapping Dictionary:")
print(atom_mapping)
"""

'import mdtraj as md\n\n# Function to create a dictionary of atom name mappings (only for CA of protein and heavy atoms of the ligand)\ndef create_atom_mapping(amber_structure, swissdock_poses):\n    print("Loading Amber and SwissDock structures...")\n    \n    # Load the entire structure (including all models) for both Amber and SwissDock poses\n    amber_traj = md.load(amber_structure)\n    swissdock_traj = md.load(swissdock_poses)\n    \n    print(f"Amber structure loaded with {amber_traj.n_atoms} atoms.")\n    print(f"SwissDock structure loaded with {swissdock_traj.n_atoms} atoms.")\n    \n    # Filter for CA atoms in the protein (only residues that are part of the protein, exclude water)\n    print("Filtering CA atoms from the Amber structure...")\n    amber_atoms = [(atom.residue.name, atom.name) for atom in amber_traj.topology.atoms\n                   if (atom.residue.name != \'HOH\' and atom.name == \'CA\')]  # Exclude water and get CA atoms\n    print(f"Found {len(amber_atoms

In [121]:
"""import mdtraj as md

# Function to create a dictionary of atom name mappings (CA atoms to CA atoms, ligand to ligand)
def create_atom_mapping(amber_structure, swissdock_poses):
    print("Loading Amber and SwissDock structures...")
    
    # Load the entire structure (including all models) for both Amber and SwissDock poses
    amber_traj = md.load(amber_structure)
    swissdock_traj = md.load(swissdock_poses)
    
    print(f"Amber structure loaded with {amber_traj.n_atoms} atoms.")
    print(f"SwissDock structure loaded with {swissdock_traj.n_atoms} atoms.")
    
    # Filter for CA atoms in the protein (only residues that are part of the protein, exclude water)
    print("Filtering CA atoms from the Amber structure...")
    amber_protein_ca_atoms = [(atom.residue.name, atom.name) for atom in amber_traj.topology.atoms
                   if (atom.residue.name != 'HOH' and atom.name == 'CA')]  # Exclude water and get CA atoms
    print(f"Found {len(amber_protein_ca_atoms)} CA atoms in the Amber structure.")
    
    # Filter for CA atoms in SwissDock structure (for protein)
    swissdock_protein_ca_atoms = [(atom.residue.name, atom.name) for atom in swissdock_traj.topology.atoms
                                  if (atom.residue.name != 'HOH' and atom.name == 'CA')]  # Exclude water and get CA atoms
    print(f"Found {len(swissdock_protein_ca_atoms)} CA atoms in the SwissDock structure.")

    # Filter for heavy atoms in the ligand (excluding hydrogen atoms) in both Amber and SwissDock
    print("Filtering heavy atoms (excluding hydrogens) for the ligand...")
    amber_ligand_atoms = [(atom.residue.name, atom.name) for atom in amber_traj.topology.atoms
                          if atom.element.symbol != 'H' and atom.residue.name != 'HOH']  # Heavy atoms excluding water
    swissdock_ligand_atoms = [(atom.residue.name, atom.name) for atom in swissdock_traj.topology.atoms
                              if atom.element.symbol != 'H' and atom.residue.name != 'HOH']  # Heavy atoms excluding water
    print(f"Found {len(amber_ligand_atoms)} heavy atoms in the Amber structure.")
    print(f"Found {len(swissdock_ligand_atoms)} heavy atoms in the SwissDock structure.")
    
    # Create atom mapping
    print("Creating atom mapping...")
    atom_mapping = {}

    # Map CA to CA (protein) atoms between Amber and SwissDock
    for (res1, atom_name1), (res2, atom_name2) in zip(amber_protein_ca_atoms, swissdock_protein_ca_atoms):
        if atom_name1 != atom_name2:
            atom_mapping[atom_name1] = atom_name2
            print(f"Protein Mapping: {atom_name1} -> {atom_name2}")
    
    # Map heavy atoms (ligand) between Amber and SwissDock
    for (res1, atom_name1), (res2, atom_name2) in zip(amber_ligand_atoms, swissdock_ligand_atoms):
        if atom_name1 != atom_name2:
            atom_mapping[atom_name1] = atom_name2
            print(f"Ligand Mapping: {atom_name1} -> {atom_name2}")
    
    print(f"Total mappings created: {len(atom_mapping)}")
    
    return atom_mapping

# Create the atom mapping dictionary using the pre-defined paths for the structures
atom_mapping = create_atom_mapping(amber_minimized_structure, swissdock_one)

# Print the atom mapping dictionary
print("Atom Mapping Dictionary:")
print(atom_mapping)
"""

'import mdtraj as md\n\n# Function to create a dictionary of atom name mappings (CA atoms to CA atoms, ligand to ligand)\ndef create_atom_mapping(amber_structure, swissdock_poses):\n    print("Loading Amber and SwissDock structures...")\n    \n    # Load the entire structure (including all models) for both Amber and SwissDock poses\n    amber_traj = md.load(amber_structure)\n    swissdock_traj = md.load(swissdock_poses)\n    \n    print(f"Amber structure loaded with {amber_traj.n_atoms} atoms.")\n    print(f"SwissDock structure loaded with {swissdock_traj.n_atoms} atoms.")\n    \n    # Filter for CA atoms in the protein (only residues that are part of the protein, exclude water)\n    print("Filtering CA atoms from the Amber structure...")\n    amber_protein_ca_atoms = [(atom.residue.name, atom.name) for atom in amber_traj.topology.atoms\n                   if (atom.residue.name != \'HOH\' and atom.name == \'CA\')]  # Exclude water and get CA atoms\n    print(f"Found {len(amber_protein_

In [122]:
# Define a dictionary
# Build a dictionary: the original name in the cluster as a label, and the value is the atom name in the other one
# This is to map the atom names from the SwissDock poses to the AMBER minimized structure
# This is necessary because the atom names may be different in the two structures
# Only include the atoms that are different in both structures

# Define the dictionary
dict_atom_names = {}

In [123]:
def map_to_undocked(contacts_docked, pdb_file_undocked):
    """
    Map docked contacts to undocked PDB file by residue and atom name matching.
    """
    traj_undocked = md.load(pdb_file_undocked)
    topology_undocked = traj_undocked.topology
    
    # Create a dictionary to store mappings
    mapped_contacts = {}
    for res1_docked, res2_docked in contacts_docked:
        # Match residues in the undocked file
        res1_match = next((res for res in topology_undocked.residues if str(res) == res1_docked), None)
        res2_match = next((res for res in topology_undocked.residues if str(res) == res2_docked), None)
        
        if res1_match and res2_match:
            mapped_contacts[(res1_docked, res2_docked)] = (str(res1_match), str(res2_match))
        else:
            mapped_contacts[(res1_docked, res2_docked)] = None  # No match found
    return mapped_contacts

# File paths
